# permits-data

Data analysis pipeline for construction permits in Los Angeles.

For more information:
https://data.lacity.org/A-Prosperous-City/Building-and-Safety-Permit-Information/yv23-pmwf

In [5]:
import os
import sys
import pandas as pd
import psycopg2

In [11]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [6]:
sys.path[0] = '../'

In [7]:
raw_data = 'permits_raw.csv'

URL = "https://data.lacity.org/api/views/yv23-pmwf/rows.csv?accessType=DOWNLOAD"
DATA_PATH = sys.path[0] + 'data/raw/' + raw_data

In [9]:
data = pd.read_csv(DATA_PATH)

/Users/gregory/anaconda3/envs/permits-data-env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,1,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
